In [1]:
import torch
import torch.nn
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
mb_size = 32
z_dim = 5
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
h_dim = 128
cnt = 0
lr = 1e-3

In [4]:
# Encoder
Q = torch.nn.Sequential(
    torch.nn.Linear(X_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, z_dim)
)

In [6]:
# Decoder
P = torch.nn.Sequential(
    torch.nn.Linear(z_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, X_dim),
    torch.nn.Sigmoid())

In [7]:
# Discriminator
D = torch.nn.Sequential(
    torch.nn.Linear(z_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, 1),
    torch.nn.Sigmoid()
)

In [8]:
# Reset Gradient
def reset_grad():
    Q.zero_grad()
    P.zero_grad()
    D.zero_grad()

In [9]:
def sample_X(size, include_y=False):
    X, y = mnist.train.next_batch(size)
    X = Variable(torch.from_numpy(X))

    if include_y:
        y = np.argmax(y, axis=1).astype(np.int)
        y = Variable(torch.from_numpy(y))
        return X, y

    return X


In [10]:
Q_solver = optim.Adam(Q.parameters(), lr=lr)
P_solver = optim.Adam(P.parameters(), lr=lr)
D_solver = optim.Adam(D.parameters(), lr=lr)


for it in range(1000000):
    X = sample_X(mb_size)

    """ Reconstruction phase """
    z_sample = Q(X)
    X_sample = P(z_sample)

    recon_loss = nn.binary_cross_entropy(X_sample, X)

    recon_loss.backward()
    P_solver.step()
    Q_solver.step()
    reset_grad()

    """ Regularization phase """
    # Discriminator
    z_real = Variable(torch.randn(mb_size, z_dim))
    z_fake = Q(X)

    D_real = D(z_real)
    D_fake = D(z_fake)

    D_loss = -torch.mean(torch.log(D_real) + torch.log(1 - D_fake))

    D_loss.backward()
    D_solver.step()
    reset_grad()

    # Generator
    z_fake = Q(X)
    D_fake = D(z_fake)

    G_loss = -torch.mean(torch.log(D_fake))

    G_loss.backward()
    Q_solver.step()
    reset_grad()

    # Print and plot every now and then
    if it % 1000 == 0:
        print('Iter-{}; D_loss: {:.4}; G_loss: {:.4}; recon_loss: {:.4}'
              .format(it, D_loss.data[0], G_loss.data[0], recon_loss.data[0]))

        samples = P(z_real).data.numpy()[:16]

        fig = plt.figure(figsize=(4, 4))
        gs = gridspec.GridSpec(4, 4)
        gs.update(wspace=0.05, hspace=0.05)

        for i, sample in enumerate(samples):
            ax = plt.subplot(gs[i])
            plt.axis('off')
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.set_aspect('equal')
            plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

        if not os.path.exists('out/'):
            os.makedirs('out/')

        plt.savefig('out/{}.png'
                    .format(str(cnt).zfill(3)), bbox_inches='tight')
        cnt += 1
        plt.close(fig)

Iter-0; D_loss: 1.439; G_loss: 0.7287; recon_loss: 0.6982
Iter-1000; D_loss: 1.371; G_loss: 0.7533; recon_loss: 0.2727
Iter-2000; D_loss: 1.449; G_loss: 0.6482; recon_loss: 0.2138
Iter-3000; D_loss: 1.408; G_loss: 0.69; recon_loss: 0.1909
Iter-4000; D_loss: 1.383; G_loss: 0.7024; recon_loss: 0.1826
Iter-5000; D_loss: 1.39; G_loss: 0.6931; recon_loss: 0.177
Iter-6000; D_loss: 1.372; G_loss: 0.7667; recon_loss: 0.1709
Iter-7000; D_loss: 1.395; G_loss: 0.7129; recon_loss: 0.1825
Iter-8000; D_loss: 1.389; G_loss: 0.6816; recon_loss: 0.1665
Iter-9000; D_loss: 1.39; G_loss: 0.6768; recon_loss: 0.1914
Iter-10000; D_loss: 1.387; G_loss: 0.6906; recon_loss: 0.1478
Iter-11000; D_loss: 1.379; G_loss: 0.7249; recon_loss: 0.167
Iter-12000; D_loss: 1.393; G_loss: 0.6823; recon_loss: 0.1833
Iter-13000; D_loss: 1.386; G_loss: 0.6821; recon_loss: 0.1486
Iter-14000; D_loss: 1.393; G_loss: 0.6952; recon_loss: 0.1572
Iter-15000; D_loss: 1.386; G_loss: 0.7; recon_loss: 0.1638
Iter-16000; D_loss: 1.383; G_l

KeyboardInterrupt: 